In [1]:
import torch
from transformers import TFBertModel, BertTokenizer

import os
import time
import pandas as pd
from datasets import load_metric
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import csv
import tensorflow as tf
import keras
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import sklearn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Input, Dense, Concatenate, Add, Dropout
from tensorflow.keras.models import Model

/home/tkhuran3/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-27 16:13:23.835498: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 16:13:23.838750: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 16:13:23.879014: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 16:13:23.879037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unab

In [2]:
model = keras.models.load_model("/home/tkhuran3/L/colbert_trained_w_features")
model.summary()

2023-11-27 16:14:10.422164: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_19 (InputLayer)       [(None, 20)]                 0         []                            
                                                                                                  
 input_20 (InputLayer)       [(None, 20)]                 0         []                            
                                                                                                  
 input_21 (InputLayer)       [(None, 20)]                 0         []                            
                                                                                                  
 input_22 (InputLayer)       [(None, 20)]                 0         []                            
                                                                                              

In [30]:
test_df= pd.read_csv("/home/tkhuran3/LOLgarithm/dataset/reddit_test_features.csv")

In [31]:
test_df.head()

,Unnamed: 0,fear,anger,anticipation,trust,surprise,positive,negative,sadness,disgust,...,repetition,sense_combination_score,farmost_path,closest_path,alliteration,max_alliteration,rhyme,max_rhyme,jokes,labels
0,0,0.000000,0.000000,0.000000,0.009009,0.000000,0.036036,0.000000,0.000000,0.000000,...,1.000000,0.893706,0.655792,1.000000,0.992425,1.000000,0.875066,0.997015,"A Family of 4, A Dad, A Mom, A Son and A Daugh...",0
1,1,0.074074,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,...,0.978674,0.996129,0.991066,0.987655,0.996389,0.957694,1.000000,0.979607,Give a man a fire and he is warm for the night...,0
2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.993228,0.986217,0.992791,0.970797,0.988942,NaN,1.000000,1.000000,"She avoided my eye contact,_____[removed]",0
3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.982541,0.997842,0.988132,1.000000,0.989010,0.993191,1.000000,No Homo_____[deleted],0
4,4,0.003660,0.003922,0.006209,0.024837,0.002549,0.038562,0.005229,0.004575,0.003268,...,0.980982,0.987449,1.000000,0.995524,0.987528,0.994414,0.980277,0.996589,Watching Childbith_____The nurse approached hi...,0


In [37]:
test_df_input = test_df.iloc[:, 1:-2]
test_df_texts = test_df.iloc[:,34:]

In [38]:
print(test_df_input.shape, test_df_texts.shape)

(2000, 33) (2000, 2)


In [39]:
test_df_texts.head()

,jokes,labels
0,"A Family of 4, A Dad, A Mom, A Son and A Daugh...",0
1,Give a man a fire and he is warm for the night...,0
2,"She avoided my eye contact,_____[removed]",0
3,No Homo_____[deleted],0
4,Watching Childbith_____The nurse approached hi...,0


In [34]:
test_df_input = test_df_input.values.tolist()
array_list = np.array(test_df_input)

In [40]:
MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

output_categories = list(test_df_texts.columns[[1]])
input_categories = list(test_df_texts.columns[[0]])

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['jokes']

output TARGET_COUNT:
	 1

output categories:
	 ['labels']


In [41]:
from transformers import BertTokenizer

MODEL_TYPE = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/tkhuran3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        sentences = sent_tokenize(row.jokes)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.jokes, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input

In [48]:
inputs = compute_input_arrays(test_df_texts, input_categories, tokenizer)

2000it [00:05, 370.56it/s]


(2000, 20)


In [51]:
print(len(inputs), len(inputs[0]), len(inputs[0][0]))
# check out input for 1st row
xx = 1
print(test_df_texts.iloc[xx,0])
print(sent_tokenize(test_df_texts.iloc[xx,0]))
inputs[0][xx], inputs[3][xx], inputs[6][xx], inputs[15][xx]

18 2000 20
Give a man a fire and he is warm for the night..._____But set a man on fire and he is warm for the rest of his life.
['Give a man a fire and he is warm for the night..._____But set a man on fire and he is warm for the rest of his life.']


(array([ 101, 2507, 1037, 2158, 1037, 2543, 1998, 2002, 2003, 4010, 2005,
        1996, 2305, 1012, 1012, 1012, 1035, 1035, 1035,  102], dtype=int32),
 array([101, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0], dtype=int32),
 array([101, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0], dtype=int32),
 array([ 101, 2507, 1037, 2158, 1037, 2543, 1998, 2002, 2003, 4010, 2005,
        1996, 2305, 1012, 1012, 1012, 1035, 1035, 1035, 1035, 1035, 2021,
        2275, 1037, 2158, 2006, 2543, 1998, 2002, 2003, 4010, 2005, 1996,
        2717, 1997, 2010, 2166, 1012,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  

In [53]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

outputs = compute_output_arrays(test_df_texts, output_categories)
outputs

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [83]:
y_pred = model.predict((inputs, array_list))

/home/tkhuran3/.local/lib/python3.10/site-packages/tensorflow/python/framework/constant_op.py:103: RuntimeWarning: overflow encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


63/63 [==============================] - 55s 605ms/step


In [88]:
y_pred = (y_pred > 0.5).astype(int)
y_pred

array([[0],
       [1],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [87]:
y_true = outputs
y_true

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [89]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

In [90]:
print('Acc', accuracy, 'Prec', precision, 'Rec', recall, 'F1',f1)

Acc 0.611 Prec 0.5817378497790869 Rec 0.79 F1 0.6700593723494487


In [91]:
from sklearn.metrics import classification_report
report = classification_report(y_true, y_pred)

In [93]:
print(report)

              precision    recall  f1-score   support

           0       0.67      0.43      0.53      1000
           1       0.58      0.79      0.67      1000

    accuracy                           0.61      2000
   macro avg       0.63      0.61      0.60      2000
weighted avg       0.63      0.61      0.60      2000



In [94]:
model_without = keras.models.load_model("/home/tkhuran3/L/colbert-trained")


In [96]:
y_pred = model_without.predict(inputs)

63/63 [==============================] - 91s 1s/step


In [98]:
y_pred = (y_pred > 0.5).astype(int)

In [99]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print('Acc', accuracy, 'Prec', precision, 'Rec', recall, 'F1',f1)

Acc 0.499 Prec 0.4994974874371859 Rec 0.994 F1 0.6648829431438127


In [101]:
report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.40      0.00      0.01      1000
           1       0.50      0.99      0.66      1000

    accuracy                           0.50      2000
   macro avg       0.45      0.50      0.34      2000
weighted avg       0.45      0.50      0.34      2000

